In [1]:
#Getting the image to be read through os liberies and adjusting the contrast for the images to be read
import cv2
import numpy as np
import os
import pandas as pd
filelist=[]
filelist_modified=[]
filelist=[file for file in os.listdir() if (file.endswith('.png') and file.startswith('Page') )]
for file in filelist:
    img=cv2.imread(file)
    increase=np.ones(img.shape,dtype='uint8')*60
    img_mod=cv2.add(img,increase)
    mod_file=file.split('.')[0]+'_modified.png'
    filelist_modified.append(mod_file)
    cv2.imwrite(mod_file,img_mod)
    cv2.destroyAllWindows()

In [4]:
# List of contrast adjusted file to be read
filelist_modified

In [11]:
#Reading image data using pytesserct and storing data like filename, lineno, sentence in a dataframe
from PIL import Image
import pytesseract as tess
final_list=[]
tess.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
for file in filelist_modified:
    counter=1
    img=Image.open(file)
    image_string=tess.image_to_string(img)
    image_list=image_string.split('\n')
    image_list.pop()
    for line in image_list:
        line=line.strip()
        if (line!=''):
            row_list=[]
            row_list.append(file.split('_')[0])
            row_list.append(counter)
            counter=counter+1
            row_list.append(line)
            final_list.append(row_list)
        else:
            continue
#Creating dataframe to store the image data with columns like filename, lineNo, Sentence from final_list
df = pd.DataFrame(final_list,columns = ['Filename','LineNo.','Sentence'])       
resultframe=df.set_index('Filename')

In [13]:
#Result dataframe with Filename as index 
resultframe.head()

,LineNo.,Sentence
Filename,,
Page-1,1,UHID: AGHL.0000265795 ~ AH - QF - WS - 14
Page-1,2,A
Page-1,3,Apollo
Page-1,4,Gleneagles HOSPITALS
Page-1,5,KOLKATA


In [16]:
#Writing data frame to excel
from pandas import ExcelWriter
writer = ExcelWriter('image_data.xlsx')
resultframe.to_excel(writer,'Sheet1')
writer.save()